In [2]:
!pip install openai

In [3]:
import module
await module._module.load('ollama')

In [4]:
import subprocess

# Start a background process using subprocess.Popen
process = subprocess.Popen(['ollama', 'serve'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

In [5]:
import nbformat

file_path = ''
nb_code = nbformat.read(file_path, as_version=4)

py_script = ''
for cell in file_path['cells']:
    if (cell['cell_type'] == 'code'):
        py_script += cell['source']
py_script

In [6]:
# file_path = "number-input.ipynb"  # Ensure this is the correct local path
# with open(file_path, 'r', encoding='utf-8') as f:
    # example1 = nbformat.read(f, as_version=4)

In [7]:
# file_path = "greetings-button.ipynb"  # Ensure this is the correct local path
# with open(file_path, 'r', encoding='utf-8') as f:
    # example2 = nbformat.read(f, as_version=4)

In [8]:
print(nb_code)

{'cells': [{'cell_type': 'code', 'execution_count': 1, 'metadata': {}, 'outputs': [], 'source': 'import ipywidgets as widgets\nfrom IPython.display import display, clear_output\n'}, {'cell_type': 'code', 'execution_count': 2, 'metadata': {}, 'outputs': [{'data': {'application/vnd.jupyter.widget-view+json': {'model_id': 'ffa82cd34d654287847eb5a960280238', 'version_major': 2, 'version_minor': 0}, 'text/plain': "HBox(children=(Button(description='☆', layout=Layout(height='40px', width='40px'), style=ButtonStyle(button_co…"}, 'metadata': {}, 'output_type': 'display_data'}, {'data': {'application/vnd.jupyter.widget-view+json': {'model_id': 'd28d4446571a48a1a4aa49718a30d2fc', 'version_major': 2, 'version_minor': 0}, 'text/plain': 'Output()'}, 'metadata': {}, 'output_type': 'display_data'}], 'source': 'sentiment_mapping = ["one", "two", "three", "four", "five"]\n\n\noutput = widgets.Output()\n\n\ndef on_button_click(selected):\n    with output:\n        clear_output()\n        print(f"You sel

In [9]:
from openai import OpenAI


In [10]:
client = OpenAI(
    base_url = 'http://localhost:11434/v1',
    api_key='ollama',
)

In [11]:
model= "deepseek-coder-v2:16b"

In [12]:
# fix the tags 
def prompt_with_examples(prompt, examples=[]):
    # Start with the initial part of the prompt
    full_prompt = "<s>[INST]\n"

    # Add each example to the prompt
    for example_prompt, example_response in examples:
        full_prompt += f"{example_prompt} [/INST] {example_response} </s><s>[INST]"

    # Add the main prompt and close the template
    full_prompt += f"{prompt} [/INST]"

    return full_prompt

In [13]:
def generate(system_prompt, examples, model):
    # Prepare the messages for the API call
    messages = [
        {
            "role": "system",
            "content": system_prompt,
        },
        {
            "role": "user",
            "content": examples,
        }
    ]

    # Call the chat completion API
    response = client.chat.completions.create(
        model=model,
        messages=messages,
     
    )

    # Extract and return the generated content
    return response.choices[0].message.content.strip()

In [14]:
example1_prompt = """
Convert this python code to streamlit code using the streamlit library.

import ipywidgets as widgets
from IPython.display import display, clear_output

sentiment_mapping = ["one", "two", "three", "four", "five"]


output = widgets.Output()


def on_button_click(selected):
    with output:
        clear_output()
        print(f"You selected {sentiment_mapping[selected]} star(s).")  

    for idx, button in enumerate(buttons):
        if idx <= selected:
            button.description = '★'  
            button.style.font_color = 'yellow' 
        else:
            button.description = '☆' 
            button.style.font_color = 'black'  

buttons = []
for i in range(5):
    button = widgets.Button(description='☆', button_style='', layout=widgets.Layout(width='40px', height='40px'))
    button.style.button_color = 'transparent'  
    button.style.font_size = '30px'  
    button.on_click(lambda b, idx=i: on_button_click(idx))  
    buttons.append(button)

star_rating_box = widgets.HBox(buttons)

with output:
    print("Please rate your experience:")

display(star_rating_box, output)
"""

example1_response = """
import streamlit as st

sentiment_mapping = ["one", "two", "three", "four", "five"]
selected = st.feedback("stars")
if selected is not None:
    st.markdown(f"You selected {sentiment_mapping[selected]} star(s).")
"""

In [15]:
example2_prompt = """
Convert this python code to streamlit code using the streamlit library.

import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
reset_button = widgets.Button(
    description='Reset',
    button_style='danger'  
)

hello_button = widgets.Button(
    description='Say hello',
    button_style=''  
)


hello_button.add_class("outlined-button")


output = widgets.Output()


def on_hello_button_click(b):
    with output:
        clear_output()  
        print("Why hello there") 


def on_reset_button_click(b):
    with output:
        clear_output()  
        print("Goodbye") 

hello_button.on_click(on_hello_button_click)
reset_button.on_click(on_reset_button_click)

with output:
    print("Goodbye")

display(reset_button, hello_button, output)

display(HTML('''
<style>
    .outlined-button {
        border: 2px solid red;  /* Red outline */
        background-color: white;  /* White background */
        color: red;  /* Red text color */
    }
</style>
'''))
"""
example2_response = """
import streamlit as st

st.button("Reset", type="primary")
if st.button("Say hello"):
    st.write("Why hello there")
else:
    st.write("Goodbye")
"""

In [16]:
system_prompt = "Streamlit code ONLY. variables should be defined.No syntax error"

In [17]:
user_prompt =  """
Convert this python code to streamlit code using the streamlit library.

import ipywidgets as widgets
from IPython.display import display

prompt = widgets.Label("Insert a Number")
number_input = widgets.FloatText(
    value=None,
    placeholder="Type a number...",
    step = 0.01
)

def on_value_change(change):
    output.clear_output()
    with output:
        print(f"The current number is {number_input.value}")

display(prompt)
display(number_input)
output = widgets.Output()
with output:
    print("The current number is None")

display(output)

number_input.observe(on_value_change, names='value')

"""

In [18]:
examples = [
    (example1_prompt, example1_response),
    (example2_prompt, example2_response)
]

In [19]:
structuredPrompt = prompt_with_examples(user_prompt, examples)
print(f"System Prompt: {system_prompt}")
print(f"Structured Prompt: {structuredPrompt}")

System Prompt: Streamlit code ONLY. variables should be defined.No syntax error
Structured Prompt: <s>[INST]

Convert this python code to streamlit code using the streamlit library.

import ipywidgets as widgets
from IPython.display import display, clear_output

sentiment_mapping = ["one", "two", "three", "four", "five"]


output = widgets.Output()


def on_button_click(selected):
    with output:
        clear_output()
        print(f"You selected {sentiment_mapping[selected]} star(s).")  

    for idx, button in enumerate(buttons):
        if idx <= selected:
            button.description = '★'  
            button.style.font_color = 'yellow' 
        else:
            button.description = '☆' 
            button.style.font_color = 'black'  

buttons = []
for i in range(5):
    button = widgets.Button(description='☆', button_style='', layout=widgets.Layout(width='40px', height='40px'))
    button.style.button_color = 'transparent'  
    button.style.font_size = '30px'  
    button.o

In [20]:
# Use the structured prompt in the generate function
print(generate(system_prompt, structuredPrompt, model= model))

```python
import streamlit as st

st.write("Insert a Number")
number = st.number_input("Type a number...", value=None, step=0.01)

if number is not None:
    st.write(f"The current number is {number}")
else:
    st.write("The current number is None")
```
